In [2]:
! pip install torchaudio soundfile tensorboard_logger
! sudo apt-get install ffmpeg

     |████████████████████████████████| 3.2MB 6.4MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [0]:
import argparse
import json
import random
import time
import os
import numpy as np
import scipy.signal

import torch
import torch.nn as nn
import torch.optim
import torchaudio
import torch.autograd as autograd
import torch.utils.data as tud
from torch.nn import CTCLoss
from torch.autograd import Variable

import tqdm
import soundfile
import tensorboard_logger as tb_log

In [4]:
libri_data = torchaudio.datasets.LIBRISPEECH('./', url='train-clean-100', folder_in_archive='LibriSpeech', download=True)

In [0]:
os.chdir('/content/LibriSpeech/')

In [0]:
# Create small datasets
! mkdir 'train-small'  'dev-small'

In [0]:
# Beta train
! gsutil -m cp -r  '/content/LibriSpeech/train-clean-100/103' '/content/LibriSpeech/train-clean-100/1034'\
 '/content/LibriSpeech/train-clean-100/1040' '/content/LibriSpeech/train-clean-100/1069' \
 '/content/LibriSpeech/train-clean-100/1081' '/content/LibriSpeech/train-clean-100/1088' \
  '/content/LibriSpeech/train-clean-100/1098' '/content/LibriSpeech/train-clean-100/1116' \
 '/content/LibriSpeech/train-small'

In [0]:
# Beta dev
! gsutil -m cp -r '/content/LibriSpeech/train-clean-100/118' '/content/LibriSpeech/train-clean-100/1183' \
 '/content/LibriSpeech/train-clean-100/1235' '/content/LibriSpeech/train-clean-100/1246' \
  '/content/LibriSpeech/train-clean-100/125' '/content/LibriSpeech/train-clean-100/1263' \
 '/content/LibriSpeech/dev-small'

In [0]:
! chmod +x '/content/LibriSpeech/flac_to_wav.sh' # giving a permision

In [0]:
os.chdir('/content/LibriSpeech/dev-small')

In [14]:
os.getcwd()

'/content/LibriSpeech/dev-small'

In [11]:
! '/content/LibriSpeech/flac_to_wav.sh'

Streaming output truncated to the last 5000 lines.
    ISFT            : Lavf57.83.100
    Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 16000 Hz, mono, s16, 256 kb/s
    Metadata:
      encoder         : Lavc57.107.100 pcm_s16le
size=     443kB time=00:00:14.17 bitrate= 256.0kbits/s speed=2.35e+03x    
video:0kB audio:443kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.017196%
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libg

In [0]:
# Train and test json files creating
import wave

def create_json(data_directory, output_file):
    labels = []
    durations = []
    keys = []
    for group in os.listdir(data_directory):
        if group.startswith('.'):
            continue
        speaker_path = os.path.join(data_directory, group)
        for speaker in os.listdir(speaker_path):
            if speaker.startswith('.'):
                continue
            labels_file = os.path.join(speaker_path, speaker,
                                       '{}-{}.trans.txt'
                                       .format(group, speaker))
            for line in open(labels_file):
                split = line.strip().split()
                file_id = split[0]
                label = ' '.join(split[1:]).lower()
                audio_file = os.path.join(speaker_path, speaker,
                                          file_id) + '.wav'
                audio = wave.open(audio_file)
                duration = float(audio.getnframes()) / audio.getframerate()
                audio.close()
                keys.append(audio_file)
                durations.append(duration)
                labels.append(label)
    with open(output_file, 'w') as out_file:
        for i in range(len(keys)):
            line = json.dumps({'key': keys[i], 'duration': durations[i],
                              'text': labels[i]})
            out_file.write(line + '\n')

In [0]:
os.chdir('/content/LibriSpeech/')

In [0]:
# Train json
create_json('/content/LibriSpeech/train-small/', 'train_small_corpus.json')

In [0]:
def array_from_wave(file_name):
    audio, samp_rate = soundfile.read(file_name, dtype='int16')
    return audio, samp_rate

def wav_duration(file_name):
    audio, samp_rate = soundfile.read(file_name, dtype='int16')
    nframes = audio.shape[0]
    duration = nframes / samp_rate
    return duration

def compute_mean_std(audio_files):
    samples = [log_specgram_from_file(af)
               for af in audio_files]
    samples = np.vstack(samples)
    mean = np.mean(samples, axis=0)
    std = np.std(samples, axis=0)
    return mean, std

def log_specgram_from_file(audio_file):
    audio, sr = array_from_wave(audio_file)
    return log_specgram(audio, sr)

def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(window_size * sample_rate / 1e3)  # calculate length of each segment
    noverlap = int(step_size * sample_rate / 1e3) # calculate number of points to overlap between segments

    # Compute a spectrogram with consecutive Fourier transforms
    _, _, spec = scipy.signal.spectrogram(audio,
                    fs=sample_rate,
                    window='hann',
                    nperseg=nperseg,
                    noverlap=noverlap,
                    detrend=False)
    return np.log(spec.T.astype(np.float32) + eps)

def read_data_json(data_json):
    with open(data_json) as fid:
        return [json.loads(l) for l in fid]

In [0]:
class Preprocessor():

    def __init__(self, data_json, max_samples=100, start_and_end=True):
        """
        Builds a preprocessor from a dataset.
        :data_json (string): A file containing a json representation
                of each example per line.
        :max_samples (int): The maximum number of examples to be used
                in computing summary statistics.
        """
        data = read_data_json(data_json)
        

        # Compute data mean, std from sample
        audio_files = [d['key'] for d in data]
        random.shuffle(audio_files)
        self.mean, self.std = compute_mean_std(audio_files[:max_samples])
        self._input_dim = self.mean.shape[0]
        

        # Make char map
        chars = list(set(t for d in data for t in d['text']))
        self.int_to_char = dict(enumerate(chars))
        self.char_to_int = {v : k for k, v in self.int_to_char.items()}
        # print('Chars: {0} \nint_to_char: {1} \nchar_to_int: {2}'.format(chars, self.int_to_char, self.char_to_int))

    def encode(self, text):
        text = list(text)
        return [self.char_to_int[t] for t in text]

    def decode(self, seq):
        text = [self.int_to_char[s] for s in seq]
        if not self.start_and_end:
            print('Decode: ', text[s:e])
            return text

        s = text[0] == self.START
        e = len(text)
        print("Before decoding: ", seq)
        print('Decoded: ', text[s:e])
        return text[s:e]

    
    def preprocess(self, wave_file, text):
        inputs = log_specgram_from_file(wave_file) # Apply log spectrogram transform to raw .wav file.
        inputs = (inputs - self.mean) / self.std # Normalize input
        targets = self.encode(text)  # Chars to ints
        return inputs, targets

    @property
    def input_dim(self):
        return self._input_dim

    @property
    def vocab_size(self):
        return len(self.int_to_char)

In [0]:
class AudioDataset(tud.Dataset):

    """
    Builds a dataset from audio samples and transcriptions.
    """

    def __init__(self, data_json, preproc, batch_size):

        data = read_data_json(data_json)
        self.preproc = preproc

        bucket_diff = 4
        # Get max transcription length in dataset 
        max_len = max(len(x['text']) for x in data)
        num_buckets = max_len // bucket_diff
        buckets = [[] for _ in range(num_buckets)]
        for d in data:
            bid = min(len(d['text']) // bucket_diff, num_buckets - 1)
            buckets[bid].append(d)

        # Sort by input length followed by output length
        sort_fn = lambda x : (round(x['duration'], 1),
                              len(x['text']))
        for b in buckets:
            b.sort(key=sort_fn)
        data = [d for b in buckets for d in b]
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_ = self.data[idx]
        data_ = self.preproc.preprocess(data_["key"],
                                        data_["text"])
        
        return data_



class BatchRandomSampler(tud.sampler.Sampler):
    """
    Batches the data consecutively and randomly samples
    by batch without replacement.
    """

    def __init__(self, data_source, batch_size):
        it_end = len(data_source) - batch_size + 1
        self.batches = [range(i, i + batch_size)
                for i in range(0, it_end, batch_size)]
        self.data_source = data_source

    def __iter__(self):
        random.shuffle(self.batches)
        return (i for b in self.batches for i in b)

    def __len__(self):
        return len(self.data_source)




In [0]:
# Create dataset loader with defined batch sampler
def make_loader(dataset_json, preproc,
                batch_size, num_workers=4):
    dataset = AudioDataset(dataset_json, preproc,
                           batch_size)
    sampler = BatchRandomSampler(dataset, batch_size)
    loader = tud.DataLoader(dataset,
                batch_size=batch_size,
                sampler=sampler,
                num_workers=num_workers,
                collate_fn=lambda batch : zip(*batch),
                drop_last=True)
    return loader

In [0]:
class Highway(nn.Module):
    def __init__(self, size, num_layers, f):

        super(Highway, self).__init__()

        # gen features from wav
        # self.spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512, window='hann', freq_scale='no', center=True, pad_mode='reflect', fmin=50,fmax=6000, sr=16000, trainable=False, output_format='Magnitude', device='cuda')
        
        self.num_layers = num_layers
        self.nonlinear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.linear = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.gate = nn.ModuleList([nn.Linear(size, size) for _ in range(num_layers)])
        self.f = f

    def forward(self, x):
        """
            :param x: tensor with shape of [batch_size, size]
            :return: tensor with shape of [batch_size, size]
            applies σ(x) ⨀ (f(G(x))) + (1 - σ(x)) ⨀ (Q(x)) transformation | G and Q is affine transformation,
            f is non-linear transformation, σ(x) is affine transformation with sigmoid non-linearition
            and ⨀ is element-wise multiplication
            """
        # x = self.spec_layer(x)

        for layer in range(self.num_layers):
            gate = torch.sigmoid(self.gate[layer](x))

            nonlinear = self.f(self.nonlinear[layer](x))
            linear = self.linear[layer](x)

            x = gate * nonlinear + (1 - gate) * linear

        return x

def add_context(X_raw, k):
    X_context = []
    for utterance in X_raw:
        n, d = utterance.shape
        utterance_with_context = torch.zeros((n, (2*k+1)*d)).cuda()
        for j in np.arange(-k, k+1, 1):
            #print(j)
            if j<0:
                utterance_with_context[:j,d*(j+k):d*(j+k+1)] = utterance[-j:]
            elif j==0:
                utterance_with_context[:,d*(j+k):d*(j+k+1)] = utterance
            else:
                utterance_with_context[j:,d*(j+k):d*(j+k+1)] = utterance[:-j]
        X_context.append(utterance_with_context.unsqueeze(0))
    return torch.cat(X_context)

In [0]:
def zero_pad_concat(inputs):
    max_t = max(inp.shape[0] for inp in inputs)
    shape = (len(inputs), max_t, inputs[0].shape[1])
    input_mat = np.zeros(shape, dtype=np.float32)
    for e, inp in enumerate(inputs):
        input_mat[e, :inp.shape[0], :] = inp
    return input_mat


# Assume input to be N_batch x T x Dim.
class myAlexa(nn.Module):
    def __init__(self, n_input, n_bottleneck = 64, n_output= 1, context = 10):
        super(myAlexa, self).__init__()
        self.encoder = Highway(n_input, 4, f=torch.nn.functional.relu)
        self.bottleneck = nn.Linear(n_input, n_bottleneck)
        self.classifier = Highway(n_bottleneck*(2*context+1), 6, f=torch.nn.functional.relu)
        self.linear = nn.Linear(n_bottleneck*(2*context+1), n_output)
        self.context = context
       
    def forward(self, x): # x: N_batch x T x Dim.
        x = torch.FloatTensor(x).cuda()
        out = self.encoder(x) # N_batch x T x Dim.
        out = self.bottleneck(out) #  N_batch x T x 64
        out = add_context(out, self.context) # N_batch x T x (64*21)
        out = self.classifier(out) # N x T x (64*21)
        out = self.linear(out) # N x T x 1
        return out # torch.FloatTensor(out.cpu()).cuda() # torch.nn.functional.softmax(out, dim=2)
        

    
    def collate(self, inputs, labels):
        max_t = max(i.shape[0] for i in inputs)
        # print("collate length inputs : ", inputs, len(inputs))
        x_lens = torch.IntTensor([max_t] * len(inputs))# .cuda()
        x = Variable(torch.FloatTensor(zero_pad_concat(inputs)))
        y_lens = torch.IntTensor([len(l) for l in labels])# .cuda()
        y = torch.IntTensor([l for label in labels for l in label]) #.cuda()
        
        
        return x, y, x_lens, y_lens

    def forward_impl(self, batch):
        x, y, x_lens, y_lens = self.collate(*batch)
        print('x size: ', x.size())
        
        # out = self.forward(x.cuda())
        
        return x, y, x_lens, y_lens
        # print(40*"=", 'forward done')
        

# __________________________________________________________________________


In [0]:
tb_log.configure('logging')

In [0]:
# tb_log.configure('logging')
random.seed(1024)
torch.manual_seed(1024)
torch.cuda.manual_seed_all(1024)

def adjust_learning_rate(optimizer, lr):
        """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
        # lr = args.lr * (0.1 ** (epoch // 30))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr


In [0]:
import editdistance
class TokenAcc():
    def __init__(self, blank=0):
        self.err = 0
        self.cnt = 0
        self.tmp_err = 0
        self.tmp_cnt = 0
        self.blank = 0
    
    def update(self, pred, xlen, label):
        ''' label is one dimensinal '''
        pred = np.vstack([pred[i, :j] for i, j in enumerate(xlen)])
        e = self._distance(pred, label)
        c = label.shape[0]
        self.tmp_err += e; self.err += e
        self.tmp_cnt += c; self.cnt += c
        return 100 * e / c

    def get(self, err=True):
        # get interval
        if err: res = 100 * self.tmp_err / self.tmp_cnt
        else: res = 100 - 100 * self.tmp_err / self.tmp_cnt
        self.tmp_err = self.tmp_cnt = 0
        return res

    def getAll(self, err=True):
        if err: return 100 * self.err / self.cnt
        else: return 100 - 100 * self.err / self.cnt

    def _distance(self, y, t):
        if len(y.shape) > 1: 
            y = np.argmax(y, axis=1)
        prev = self.blank
        hyp = []
        for i in y:
            if i != self.blank and i != prev: hyp.append(i)
            prev = i
        return editdistance.eval(hyp, t)

In [0]:

model = myAlexa(161)
model.cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=.9)
criterion = CTCLoss()
batch_size = 4
lr = 1e-5
n_epochs = 10

# data set


preproc = Preprocessor("/content/LibriSpeech/train_small_corpus.json",
start_and_end=True)


train_ldr = make_loader( "/content/LibriSpeech/train_small_corpus.json",
                    preproc, batch_size)


dev_ldr = make_loader( "/content/LibriSpeech/dev_small_corpus.json",
                    preproc, batch_size)



In [0]:
tri = cvi = 0
def eval():
    global cvi
    losses = []
    tacc = TokenAcc()
    for i, batch in enumerate(dev_ldr):
      
        xs, ys, xlen, ylen = model.forward_impl(batch)
        
    
        model.eval()
        out = model(xs)
        loss = criterion(out.transpose(0,1).cpu(), ys, xlen, ylen)
        loss = float(loss.data) * len(xlen) # batch size
        losses.append(loss)
        tacc.update(out.data.cpu().numpy(), xlen, ys)
        tb_log.log_value('cv_loss', loss/len(xlen), cvi)
        cvi += 1
    return sum(losses) / len(devset), tacc.getAll()

def train(lr=0):
    
    global tri
    prev_loss = 1000
    best_model = None
    lr = lr
    for epoch in range(1, n_epochs+1):
        totloss = 0; losses = []
        start_time = time.time()
        tacc = TokenAcc()
        for i, batch in enumerate(train_ldr):

            xs, ys, xlen, ylen = model.forward_impl(batch)
            

            model.train()
            optimizer.zero_grad()
            out = model(xs)

            loss = criterion(out.transpose(0,1).cpu(), ys, xlen, ylen)
            print(loss.data)
            loss.backward()
            loss = float(loss.data) * len(xlen) # batch size
            totloss += loss; losses.append(loss)
            tacc.update(out.data.cpu().numpy(), xlen, ys)
            grad_norm = nn.utils.clip_grad_norm(model.parameters(), 200)
            optimizer.step()

            tb_log.log_value('train_loss', loss/len(xlen), tri)
            tb_log.log_value('train_grad_norm', grad_norm, tri)
            tri += 1

            if i % 50 == 0 and i > 0:
                loss = totloss / batch_size / 50
                logging.info('[Epoch %d Batch %d] loss %.2f, PER %.2f'%(epoch, i, loss, tacc.get()))
                totloss = 0

        losses = sum(losses) / len(trainset)
        val_l, per = eval()
        logging.info('[Epoch %d] time cost %.2fs, train loss %.2f, PER %.2f; cv loss %.2f, PER %.2f; lr %.3e'%(
            epoch, time.time()-start_time, losses, tacc.getAll(), val_l, per, lr
        ))

        if val_l < prev_loss:
            prev_loss = val_l
            best_model = '{}/params_epoch{:02d}_tr{:.2f}_cv{:.2f}'.format('best_model', epoch, losses, val_l)
            torch.save(model.state_dict(), best_model)
        else:
            torch.save(model.state_dict(), '{}/params_epoch{:02d}_tr{:.2f}_cv{:.2f}_rejected'.format(args.out, epoch, losses, val_l))
            model.load_state_dict(torch.load(best_model))
            model.cuda()
            lr /= 2
            adjust_learning_rate(optimizer, lr)

In [33]:
train(lr)

x size:  torch.Size([4, 1604, 161])
tensor(-6.4710e+20)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


x size:  torch.Size([4, 1520, 161])
tensor(-4.6153)
x size:  torch.Size([4, 1597, 161])
tensor(-2.8987)
x size:  torch.Size([4, 1526, 161])
tensor(-0.3616)
x size:  torch.Size([4, 1546, 161])
tensor(4.9337)
x size:  torch.Size([4, 1503, 161])
tensor(6.9413)
x size:  torch.Size([4, 1094, 161])
tensor(10.9304)
x size:  torch.Size([4, 1553, 161])
tensor(15.4226)
x size:  torch.Size([4, 1299, 161])
tensor(18.2593)
x size:  torch.Size([4, 635, 161])
tensor(-1.0348e+27)
x size:  torch.Size([4, 1108, 161])
tensor(9.8748)
x size:  torch.Size([4, 1611, 161])
tensor(11.7183)
x size:  torch.Size([4, 1517, 161])
tensor(13.7161)
x size:  torch.Size([4, 1064, 161])
tensor(13.2370)
x size:  torch.Size([4, 1522, 161])
tensor(8.6444)
x size:  torch.Size([4, 1591, 161])
tensor(12.0937)
x size:  torch.Size([4, 502, 161])
tensor(9.5201)
x size:  torch.Size([4, 1268, 161])
tensor(10.9494)
x size:  torch.Size([4, 1445, 161])
tensor(16.7582)
x size:  torch.Size([4, 1313, 161])
tensor(10.1240)
x size:  torch.

Process Process-6:
Process Process-5:
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
Process Process-8:
Process Process-7:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _r

KeyboardInterrupt: ignored

In [0]:
torch.cuda.is_available()

In [0]:
import torch
torch.cuda.empty_cache()

In [1]:
! nvidia-smi

Mon Jun 15 10:45:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------